# 4 - Results

In [58]:
import pandas as pd

In [59]:
dataset_file = "dataset/gdpr-br-qa.jsonl"
df_dataset = pd.read_json(dataset_file, lines=True, encoding='utf-8')

In [60]:
EXPERIMENTS = [
    "no_context_gpt-4o-mini-2024-07-18",
    "rag_3_gpt-4o-mini-2024-07-18",
    "with_gdpr_gpt-4o-mini-2024-07-18",
    
    "no_context_llama-3.3-70b-versatile",
    "rag_3_llama-3.3-70b-versatile",
    "with_gdpr_llama-3.3-70b-versatile",
    
    "no_context_deepseek-chat",
    "rag_3_deepseek-chat",
    "with_gdpr_deepseek-chat",
    
    "no_context_sabia-3-2024-12-11",
    "rag_3_sabia-3-2024-12-11",
    "with_gdpr_sabia-3-2024-12-11",
]

POSSIBLE_ANSWERS = ["A", "B", "C", "D", "E", "Certo", "Errado"]

In [68]:
def calcula_aderencia_ao_prompt(df):
    df_aderencia = df.copy()
    
    for exp in EXPERIMENTS:
        df_aderencia[exp] = df[exp].isin(POSSIBLE_ANSWERS)

    return df_aderencia

In [69]:
def calcular_resultados(df):
    df_result = df.copy()
    
    for exp in EXPERIMENTS:
        df_result[exp] = df['Answer'] == df[exp]

    return df_result

In [70]:
def normalizar_resposta_string(valor):
    # Se o valor for um dos possíveis, está certo e não faz nada
    if valor in POSSIBLE_ANSWERS:
        return valor

    # Se a resposta começar com algumas frases, remove essas frases:
    frases_para_remover = ["A resposta correta é a alternativa: ",
                           "A resposta correta é a alternativa ",
                           "A resposta é "]
    for para_remover in frases_para_remover:
        if valor.startswith(para_remover):
            valor = valor[len(para_remover):]
        
    # Tenta tratar o caso em que o LLM copia a resposta (por exemplo: "A) Apenas I, II e III").
    # E o caso dele colocar uma resposta possível e botar um ponto final (por exemplo: "Errado.")
    # Nesses casos, retorna só o valor da resposta
    for ans in POSSIBLE_ANSWERS:
        if valor.startswith(ans + ".") or valor.startswith(ans + ")"):
            return ans  # Retorna o valor base encontrado em POSSIBLE_ANSWERS

    # Se não cair nas regras acima, retorna do jeito errado mesmo
    return valor
        
def normalizar_respostas(df):
    df_tratado = df.copy()  # Copia a coluna ID
    
    #Aplicando a normalização para cada coluna de EXPERIMENTS
    for exp in EXPERIMENTS:
        df_tratado[exp] = df[exp].apply(normalizar_resposta_string)
    
    return df_tratado

In [72]:
print('########### ADERÊNCIA AO PROMPT ###########')
df_aderencia = calcula_aderencia_ao_prompt(df_dataset)
df_aderencia[EXPERIMENTS].mean()

########### ADERÊNCIA AO PROMPT ###########


no_context_gpt-4o-mini-2024-07-18     0.992857
rag_3_gpt-4o-mini-2024-07-18          0.988571
with_gdpr_gpt-4o-mini-2024-07-18      0.938571
no_context_llama-3.3-70b-versatile    0.904286
rag_3_llama-3.3-70b-versatile         0.934286
with_gdpr_llama-3.3-70b-versatile     0.301429
no_context_deepseek-chat              1.000000
rag_3_deepseek-chat                   1.000000
with_gdpr_deepseek-chat               1.000000
no_context_sabia-3-2024-12-11         0.977143
rag_3_sabia-3-2024-12-11              0.978571
with_gdpr_sabia-3-2024-12-11          0.842857
dtype: float64

In [73]:
print('########### ADERÊNCIA AO PROMPT - APÓS NORMALIZAÇÃO DOS RESULTADOS ###########')
df_aderencia = calcula_aderencia_ao_prompt(normalizar_respostas(df_dataset))
df_aderencia[EXPERIMENTS].mean()

########### ADERÊNCIA AO PROMPT - APÓS NORMALIZAÇÃO DOS RESULTADOS ###########


no_context_gpt-4o-mini-2024-07-18     1.000000
rag_3_gpt-4o-mini-2024-07-18          0.998571
with_gdpr_gpt-4o-mini-2024-07-18      0.995714
no_context_llama-3.3-70b-versatile    0.984286
rag_3_llama-3.3-70b-versatile         0.965714
with_gdpr_llama-3.3-70b-versatile     0.972857
no_context_deepseek-chat              1.000000
rag_3_deepseek-chat                   1.000000
with_gdpr_deepseek-chat               1.000000
no_context_sabia-3-2024-12-11         1.000000
rag_3_sabia-3-2024-12-11              1.000000
with_gdpr_sabia-3-2024-12-11          1.000000
dtype: float64

In [75]:
print('########### RESULTADOS SEM TRATAR A RESPOSTA ###########')
df_results = calcular_resultados(df_dataset)
df_results[EXPERIMENTS].mean()

########### RESULTADOS SEM TRATAR A RESPOSTA ###########


no_context_gpt-4o-mini-2024-07-18     0.757143
rag_3_gpt-4o-mini-2024-07-18          0.838571
with_gdpr_gpt-4o-mini-2024-07-18      0.832857
no_context_llama-3.3-70b-versatile    0.731429
rag_3_llama-3.3-70b-versatile         0.825714
with_gdpr_llama-3.3-70b-versatile     0.277143
no_context_deepseek-chat              0.850000
rag_3_deepseek-chat                   0.905714
with_gdpr_deepseek-chat               0.941429
no_context_sabia-3-2024-12-11         0.867143
rag_3_sabia-3-2024-12-11              0.884286
with_gdpr_sabia-3-2024-12-11          0.780000
dtype: float64

In [76]:
print('########### RESULTADOS TRATANDO A RESPOSTA ###########')
df_results = calcular_resultados(normalizar_respostas(df_dataset))
df_results[EXPERIMENTS].mean()

########### RESULTADOS TRATANDO A RESPOSTA ###########


no_context_gpt-4o-mini-2024-07-18     0.761429
rag_3_gpt-4o-mini-2024-07-18          0.841429
with_gdpr_gpt-4o-mini-2024-07-18      0.862857
no_context_llama-3.3-70b-versatile    0.784286
rag_3_llama-3.3-70b-versatile         0.847143
with_gdpr_llama-3.3-70b-versatile     0.871429
no_context_deepseek-chat              0.850000
rag_3_deepseek-chat                   0.905714
with_gdpr_deepseek-chat               0.941429
no_context_sabia-3-2024-12-11         0.887143
rag_3_sabia-3-2024-12-11              0.902857
with_gdpr_sabia-3-2024-12-11          0.918571
dtype: float64

In [ ]:
df_dataset.columns